In [1]:
#imports
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from pycaret import *
from pycaret.classification import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import (classification_report)
from sklearn.model_selection import cross_val_score, train_test_split

In [ ]:
dados = pd.read_csv('Dataset_cliente.csv')
dados.head()

In [ ]:
dados.columns

In [9]:
# colunas_selecionadas = dados[['Interresses3', 'Interesses2', 'Interesses']]  
# dados['Interesses'] = colunas_selecionadas.apply(lambda row: [row["Interresses3"], row["Interesses2"], row["Interesses"]], axis=1)
# dados = dados.drop(['Interresses3', 'Interesses2'], axis=1)
# dados.to_csv("Dataset_cliente.csv")

In [10]:
# dados['Viaja_Frequentemente'] = dados['Viaja_Frequentemente'].replace({"Sim": 1, "Não": 0})
# dados['Dependentes'] = dados['Dependentes'].replace({"1\"": 1, "0\"": 0, "2\"": 2, "3\"": 3})
# dados.to_csv("Dataset_cliente.csv")

In [ ]:
def verificar_idade(idade):
    if 18 <= idade <=29:
        return "Plano Conexão"
    elif 30 <= idade < 50:
        return "Plano Serenidade"
    else:
        return "Plano Equilíbrio"

dados['Data_Nascimento'] = pd.to_datetime(dados['Data_Nascimento'])
dados['Idade'] = datetime.date.today().year - dados['Data_Nascimento'].dt.year
dados['Plano'] = dados['Idade'].apply(verificar_idade)


dados[['Idade', 'Plano']]


In [ ]:
especificacao = dados[['Interesses', 'Gastos_Mensais', 'Salario',
       'Tipo_Cartao_Credito', 'Gasto_Mensal_Cartao', 'Viaja_Frequentemente',
       'Profissao', 'Renda_Mensal', 'Dependentes', 'Idade', 'Plano']]
especificacao.head(5)

In [ ]:
def defineplano(interesses):
    if 'tecnologia' in interesses or 'Viagens' in interesses or 'Games' in interesses:
        return "Plano Conexão"
    elif 'Design' in interesses or 'Culinária' in interesses or 'Nutrição' in interesses:
        return "Plano Serenidade"
    else:
        return "Plano Equilíbrio"

# dados['Data_Nascimento'] = pd.to_datetime(dados['Data_Nascimento'])
# dados['Idade'] = datetime.date.today().year - dados['Data_Nascimento'].dt.year
dados['Plano'] = dados['Interesses'].apply(defineplano)
dados[['Interesses', 'Plano']]

In [ ]:
dados.head()

In [ ]:
e = especificacao[['Gastos_Mensais', 'Salario','Gasto_Mensal_Cartao', 'Viaja_Frequentemente','Renda_Mensal', 'Dependentes']]
#e['Viaja_Frequentemente'] = e['Viaja_Frequentemente'].str.replace("Sim", 1, case=False, regex=False)
#e['Dependentes'] = e['Dependentes'].str.replace("0\"", "0", case=False, regex=False)
e['Viaja_Frequentemente'] = e['Viaja_Frequentemente'].replace({"Sim": 1, "Não": 0})
e['Dependentes'] = e['Dependentes'].replace({"1\"": 1, "0\"": 0, "2\"": 2, "3\"": 3})
e.head()

In [ ]:
especificacao.hist(bins=100, figsize=(15, 15))
plt.show()

In [ ]:
especificacao.plot(kind='box', subplots=True, layout=(len(especificacao.columns),len(especificacao.columns)), sharex=False, sharey=False, figsize=(15, 15))
plt.show()

In [7]:
entradas = especificacao[['Gastos_Mensais', 'Salario',
       'Gasto_Mensal_Cartao', 'Viaja_Frequentemente',
       'Renda_Mensal', 'Dependentes', 'Idade']]
classes = especificacao['Plano']

In [ ]:
print(f"Formato das tabelas de dados {entradas.shape} e classes {classes.shape}")

entradas_treino, entradas_teste, classes_treino, classes_teste = train_test_split(entradas, classes, test_size=0.2)

print(f"Formato das tabelas de dados de treino {entradas_treino.shape} e teste {entradas_teste.shape}")

In [ ]:

from sklearn.neighbors import KNeighborsClassifier

k = 12
modelo = KNeighborsClassifier(n_neighbors=k)

modelo.fit(entradas_treino, classes_treino)

In [ ]:
classes_encontradas = modelo.predict(entradas_teste)
print("Predição: {}".format(classes_encontradas))

In [ ]:
from sklearn.metrics import accuracy_score
acertos = accuracy_score(classes_teste, classes_encontradas)
print("Acerto médio de classificação: ", acertos)

In [52]:
df_treino = pd.concat((entradas_treino, classes_treino), axis=1)

novas_classes = pd.Series(classes_encontradas, name="species", index=entradas_teste.index)
df_teste = pd.concat((entradas_teste, novas_classes), axis=1)

In [ ]:
k_range = list(range(1,20))
acertos = []
for k in k_range:
    modelo = KNeighborsClassifier(n_neighbors=k)
    modelo.fit(entradas_treino, classes_treino)
    classes_encontradas = modelo.predict(entradas_teste)
    acertos.append(accuracy_score(classes_teste, classes_encontradas))


plt.plot(k_range, acertos)
plt.xlabel('Valor de k do KNN')
plt.ylabel('Taxa de acertos')
plt.title('Taxa de acertos x valor de k do KNN')
plt.show()


In [ ]:
dados = pd.read_csv("Dataset_cliente.csv")
X = entradas
y = classes

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

lda_model = LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                           priors=None, shrinkage=None, solver='svd',
                           store_covariance=False, tol=0.0001)
lda_model.fit(X_train, y_train)


scores = cross_val_score(lda_model, X, y, cv=5, scoring='accuracy')
print(f"Cross-validation scores: {scores}")
print(f"Mean cross-validation score: {np.mean(scores)}")

In [ ]:
y_pred = lda_model.predict(X_test)
print("Relatório de Classificação para LDA:")
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


entradas_processadas = pd.get_dummies(entradas)
label_encoder = LabelEncoder()
classes_processadas = label_encoder.fit_transform(classes)
X_train, X_test, y_train, y_test = train_test_split(entradas_processadas, classes_processadas, test_size=0.2, random_state=42)
modelo = DecisionTreeClassifier()
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

acuracia = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {acuracia}')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_train, y_train)

y_pred_rf = modelo_rf.predict(X_test)

acuracia_rf = accuracy_score(y_test, y_pred_rf)
print(f'Acurácia do modelo Random Forest: {acuracia_rf}')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

acuracia = accuracy_score(y_test, y_pred)
precisao = precision_score(y_test, y_pred, average='weighted')
revocacao = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
matriz_confusao = confusion_matrix(y_test, y_pred)
#roc_auc = roc_auc_score(y_test, y_pred, average='weighted', multi_class='ovr')

print(f'Acurácia: {acuracia}')
print(f'Precisão: {precisao}')
print(f'Revocação: {revocacao}')
print(f'F1-Score: {f1}')
print(f'Matriz de Confusão:\n{matriz_confusao}')
#print(f'ROC-AUC: {roc_auc}')

In [ ]:
from pycaret.classification import *

clf1 = setup(data=especificacao, target='Plano', session_id=123)


best_model = compare_models()


save_model(best_model, 'modelo_melhor_salvo')